# Suspicious Control Panel DLL Load
Detects suspicious Rundll32 execution from control.exe as used by Equation Group and Exploit Kits

## Rule Content
```
- title: Suspicious Control Panel DLL Load
  id: d7eb979b-c2b5-4a6f-a3a7-c87ce6763819
  status: experimental
  description: Detects suspicious Rundll32 execution from control.exe as used by Equation
    Group and Exploit Kits
  author: Florian Roth
  date: 2017/04/15
  references:
  - https://twitter.com/rikvduijn/status/853251879320662017
  tags:
  - attack.defense_evasion
  - attack.t1073
  - attack.t1085
  - car.2013-10-002
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      ParentImage: '*\System32\control.exe'
      CommandLine: '*\rundll32.exe *'
    filter:
      CommandLine: '*Shell32.dll*'
    condition: selection and not filter
  fields:
  - CommandLine
  - ParentCommandLine
  falsepositives:
  - Unknown
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((process_parent_path.keyword:*\\System32\\control.exe AND process_command_line.keyword:*\\rundll32.exe\ *) AND (NOT (process_command_line.keyword:*Shell32.dll*)))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()